In [170]:
import pandas as pd
import numpy as np
import re
import polars
import emoji
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize,sent_tokenize
import contractions

# Simple EDA

In [245]:
data = {
    "Review_ID": range(1, 21),
    "Review_Text": [
        "The PRODUCT quality is EXCELLENT!!! 😍 Bought on 12-01-2024 at 10:30AM",
        "Worst experience EVER!!! 😡 totally disappointed... visit http://badreviews.com",
        "The service was OKAY, nothing special #average",
        "I LOVE this product <b>Highly Recommended</b> 👍👍",
        "Very poor customer support @supportteam, delayed delivery 😞",
        "<p>Average performance</p> could be BETTER!!!",
        "Absolutely FANTASTIC!!! 😍🔥 #HappyCustomer",
        "Not worth the money I paid 5000₹ 😠",
        "The app works fine BUT sometimes crashes @devteam",
        "Great VALUE for money 💰 Fast delivery on 2024-05-10",
        "Terrible packaging!!! damaged product 😡😡",
        "The product does WHAT it claims <xml>verified</xml>",
        "Amazing experience 😊 Will BUY again!!!",
        "Completely USELESS and frustrating 😤😤",
        "Neither good NOR bad, just okay... 🤷",
        "Customer service was VERY helpful & polite @helpdesk",
        "Bad quality material!!! broke within 3 days 😞",
        "The design is decent BUT functionality is LIMITED #design",
        "Extremely satisfied with the purchase 😍 #Satisfied",
        "Regret buying this 😡 Very disappointing!!! OrderID: 12345"
    ],
    "Sentiment": [
        "Positive", "Negative", "Neutral", "Positive", "Negative",
        "Neutral", "Positive", "Negative", "Neutral", "Positive",
        "Negative", "Neutral", "Positive", "Negative", "Neutral",
        "Positive", "Negative", "Neutral", "Positive", "Negative"
    ],
    "Email_ID": [
        "vamshi.kumar@gmail.com",
        "rockstar_99@yahoo.com",
        "data.learner@gmail.com",
        "ml.engineer@outlook.com",
        "support.user@yahoo.com",
        "analytics.pro@gmail.com",
        "happy.customer@gmail.com",
        "budget.buyer@yahoo.com",
        "app.tester@outlook.com",
        "fast.delivery@gmail.com",
        "angry.user@yahoo.com",
        "verified.buyer@outlook.com",
        "repeat.customer@gmail.com",
        "frustrated.user@yahoo.com",
        "neutral.reviewer@gmail.com",
        "polite.customer@outlook.com",
        "quality.issue@yahoo.com",
        "design.reviewer@gmail.com",
        "satisfied.user@outlook.com",
        "disappointed.buyer@yahoo.com"
    ]
}

df = pd.DataFrame(data)
df


,Review_ID,Review_Text,Sentiment,Email_ID
0,1,The PRODUCT quality is EXCELLENT!!! 😍 Bought o...,Positive,vamshi.kumar@gmail.com
1,2,Worst experience EVER!!! 😡 totally disappointe...,Negative,rockstar_99@yahoo.com
2,3,"The service was OKAY, nothing special #average",Neutral,data.learner@gmail.com
3,4,I LOVE this product <b>Highly Recommended</b> 👍👍,Positive,ml.engineer@outlook.com
4,5,"Very poor customer support @supportteam, delay...",Negative,support.user@yahoo.com
5,6,<p>Average performance</p> could be BETTER!!!,Neutral,analytics.pro@gmail.com
6,7,Absolutely FANTASTIC!!! 😍🔥 #HappyCustomer,Positive,happy.customer@gmail.com
7,8,Not worth the money I paid 5000₹ 😠,Negative,budget.buyer@yahoo.com
8,9,The app works fine BUT sometimes crashes @devteam,Neutral,app.tester@outlook.com
9,10,Great VALUE for money 💰 Fast delivery on 2024-...,Positive,fast.delivery@gmail.com


# Check whether the text is in lower or upper or (lower and upper) 

In [19]:
# check  whether the data is in upper or lower
df['Review_Text'].apply(lambda x: True if x.islower() or x.isupper() else False).sum()

np.int64(0)

In [20]:
# check whether the data is a combination of upper and lower
df['Review_Text'].apply(lambda x: False if x.islower() or x.isupper() else True).sum()

np.int64(20)

# Check whether the text contains any xml or html tags

In [21]:
df['Review_Text'].apply(lambda x:True if re.search(r'<.*?>',x) else False).sum()

np.int64(3)

# Check whether the text contains urls

In [22]:
df['Review_Text'].apply(lambda x:True if re.search(r'http[s]?://\S+',x) else False).sum()

np.int64(1)

# Check whether the text data contains any email ids

In [23]:
df['Email_ID'].apply(lambda x: True if re.search(r'\S+@\S+',x) else False).sum()

np.int64(20)

# Check whether the text data contains any mentions/hastags

In [28]:
df['Review_Text'].apply(lambda x: True if re.search(r'\B[@#]\S+',x) else False).sum()

np.int64(7)

# Check whether the text data contains any emojis

In [31]:
import emoji

In [32]:
text="Absolutely FANTASTIC!!! 😍🔥 #HappyCustomer"

In [33]:
emoji.emoji_count(text)

2

In [34]:
emoji.demojize(text)

'Absolutely FANTASTIC!!! :smiling_face_with_heart-eyes::fire: #HappyCustomer'

In [39]:
df['Review_Text'].apply(lambda x: True if emoji.emoji_count(x) else False).sum()

np.int64(14)

# Check whether the text data contains any numbers

In [40]:
df['Review_Text'].apply(lambda x: True if re.search('\d',x) else False).sum()

np.int64(5)

# Check whether the text data contains any punctuations

In [41]:
from string import punctuation
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [43]:
df['Review_Text'].apply(lambda x: True if re.search('[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]',x)else False).sum()

np.int64(18)

# Check whether the text data contains any date

In [50]:
df['Review_Text'].apply(lambda x: True if re.findall(r'\d{1,2}-\d{1,2}-\d{4}', x) else False).sum()

np.int64(1)

# Simple Eda function

In [246]:
def simple_eda(data,column):
    import emoji
    lower_upper=data[column].apply(lambda x: False if x.islower() or x.isupper() else True).sum()
    html_xml=data[column].apply(lambda x:True if re.search(r'<.*?>',x) else False).sum()
    urls=data[column].apply(lambda x:True if re.search(r'http[s]?://\S+',x) else False).sum()
    emails=data[column].apply(lambda x: True if re.search(r'\S+@\S+',x) else False).sum()
    mentions_hastags=data[column].apply(lambda x: True if re.search(r'\B[@#]\S+',x) else False).sum()
    emo=data[column].apply(lambda x: True if emoji.emoji_count(x) else False).sum()
    digits=data[column].apply(lambda x: True if re.search('\d',x) else False).sum()
    punctuations=data[column].apply(lambda x: True if re.search('[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]',x)else False).sum()
    date=data[column].apply(lambda x: True if re.findall(r'\d{1,2}-\d{1,2}-\d{4}', x) else False).sum()
    missing_values=data[column].isna().sum()
    duplicates=data[column].duplicated().sum()

    if lower_upper>0:
        print(f'The {column} contains the both lower and upper cases')
    if html_xml>0:
        print(f'The {column} contains the xml or html tags')
    if urls>0:
        print(f'The {column} contains urls')
    if emails>0:
        print(f'The {column} contains emails')
    if mentions_hastags>0:
        print(f'The {column} contains mentions and hastags')
    if emo>0:
        print(f'The {column} contains emojis')
    if digits>0:
        print(f'The {column} contains digits')
    if punctuations>0:
        print(f'The {column} contains Punctuations')
    if date>0:
        print(f'The {column} contains date')
    if missing_values>0:
        print(f'The {column} has {missing_values} missing values')
    if duplicates>0:
        print(f'The {column} has {duplicates} duplicate values')
    

In [60]:
data=pd.read_csv(r"C:\Users\masir\Downloads\Machine Learning\NLP\train (2).csv",on_bad_lines='skip',delimiter=';')
data

,Unnamed: 0,title,text,label
0,0,Palestinians switch off Christmas lights in Be...,"RAMALLAH, West Bank (Reuters) - Palestinians s...",1
1,1,China says Trump call with Taiwan president wo...,BEIJING (Reuters) - U.S. President-elect Donal...,1
2,2,FAIL! The Trump Organization’s Credit Score W...,While the controversy over Trump s personal ta...,0
3,3,Zimbabwe military chief's China trip was norma...,BEIJING (Reuters) - A trip to Beijing last wee...,1
4,4,THE MOST UNCOURAGEOUS PRESIDENT EVER Receives ...,There has never been a more UNCOURAGEOUS perso...,0
...,...,...,...,...
24348,24348,Mexico Senate committee OK's air transport dea...,MEXICO CITY (Reuters) - A key committee in Mex...,1
24349,24349,BREAKING: HILLARY CLINTON’S STATE DEPARTMENT G...,IF SHE S NOT TOAST NOW THEN WE RE IN BIGGER TR...,0
24350,24350,trump breaks from stump speech to admire beaut...,kremlin nato was created for agression \nruss...,0
24351,24351,NFL PLAYER Delivers Courageous Message: Stop B...,Dallas Cowboys star wide receiver Dez Bryant t...,0


In [63]:
simple_eda(data,'text')

The text contains the both lower and upper cases
The text contains the xml or html tags
The text contains urls
The text contains emails
The text contains mentions and hastags
The text contains emojis
The text contains digits
The text contains Punctuations
The text contains date


# Data preprocessing

# Convert data into lower case or upper case (based on problem statment)

In [66]:
data=pd.DataFrame({'Reviews':['I LOVE BIRYANI','I Love Biryani','i love biryani']})

In [67]:
data

,Reviews
0,I LOVE BIRYANI
1,I Love Biryani
2,i love biryani


In [68]:
data['Reviews'].str.lower()

0    i love biryani
1    i love biryani
2    i love biryani
Name: Reviews, dtype: str

In [69]:
data['Reviews'].str.upper()

0    I LOVE BIRYANI
1    I LOVE BIRYANI
2    I LOVE BIRYANI
Name: Reviews, dtype: str

# remove the tags

In [73]:
data=pd.DataFrame({'Reviews':['<p>I LOVE BIRYANI</p>','<span><p>I Love Biryani</p></span>','<div>i love biryani</div>']})

In [74]:
data

,Reviews
0,<p>I LOVE BIRYANI</p>
1,<span><p>I Love Biryani</p></span>
2,<div>i love biryani</div>


In [75]:
data['Reviews'].apply(lambda x: re.sub(r'<.*?>',' ',x))

0       I LOVE BIRYANI 
1      I Love Biryani  
2       i love biryani 
Name: Reviews, dtype: str

# remove any urls

In [79]:
data = pd.DataFrame({
    'Reviews': [
        'I LOVE BIRYANI i have order from https://swiggy',
        'I Love Biryani i have orderd from https://Zomato',
        'i love biryani i have orderd from http://pizza'
    ]
})

data


,Reviews
0,I LOVE BIRYANI i have order from https://swiggy
1,I Love Biryani i have orderd from https://Zomato
2,i love biryani i have orderd from http://pizza


In [80]:
data['Reviews'].apply(lambda x:re.sub(r'https?://\S+','',x))

0     I LOVE BIRYANI i have order from 
1    I Love Biryani i have orderd from 
2    i love biryani i have orderd from 
Name: Reviews, dtype: str

In [90]:
data = pd.DataFrame({
    'Reviews': [
        'I LOVE BIRYANI i have order from https:\\swiggy contact me at user1@gmail.com',
        'I Love Biryani i have orderd from https:\\ mail id is foodie@yahoo.com',
        'i love biryani i have orderd from http:\\pizza reach me at test_user@outlook.com'
    ]
})

data


,Reviews
0,I LOVE BIRYANI i have order from https:\swiggy...
1,I Love Biryani i have orderd from https:\ mail...
2,i love biryani i have orderd from http:\pizza ...


In [91]:
data['Reviews']=data['Reviews'].apply(lambda x:re.sub(r'\S+@+\S+','',x))

In [92]:
data.loc[0,'Reviews']

'I LOVE BIRYANI i have order from https:\\swiggy contact me at '

# remove mentions and hashtags

In [93]:
data = pd.DataFrame({
    'Reviews': [
        'I love biryani  @swiggy_support #food',
        'I LOVE biryani  @zomato_care #biryani',
        'i love biryani  @pizza_help #hungry'
    ]
})
data

,Reviews
0,I love biryani @swiggy_support #food
1,I LOVE biryani @zomato_care #biryani
2,i love biryani @pizza_help #hungry


In [94]:
data['Reviews'].apply(lambda x: re.sub(r'\B[@#]\S+','',x))

0    I love biryani   
1    I LOVE biryani   
2    i love biryani   
Name: Reviews, dtype: str

# convert Images

In [96]:
import emoji

In [95]:

df = pd.DataFrame({
    "Review_Text": [
        "I love this product 😍🔥 totally worth it!",
        "Worst experience ever 😡👎 very disappointed",
        "It is okay 🙂 nothing special but works fine",
        "Amazing quality and fast delivery 🚀😊",
        "Not happy with the service 😞😤 will not recommend"
    ]
})

df


,Review_Text
0,I love this product 😍🔥 totally worth it!
1,Worst experience ever 😡👎 very disappointed
2,It is okay 🙂 nothing special but works fine
3,Amazing quality and fast delivery 🚀😊
4,Not happy with the service 😞😤 will not recommend


In [101]:
df['Review_Text'].apply(lambda x:emoji.demojize(x,delimiters=('','')))

0    I love this product smiling_face_with_heart-ey...
1    Worst experience ever enraged_facethumbs_down ...
2    It is okay slightly_smiling_face nothing speci...
3    Amazing quality and fast delivery rocketsmilin...
4    Not happy with the service disappointed_facefa...
Name: Review_Text, dtype: str

# Stopwords

In [2]:
from nltk.corpus import stopwords

In [3]:
stopwords.fileids()

['albanian',
 'arabic',
 'azerbaijani',
 'basque',
 'belarusian',
 'bengali',
 'catalan',
 'chinese',
 'danish',
 'dutch',
 'english',
 'finnish',
 'french',
 'german',
 'greek',
 'hebrew',
 'hinglish',
 'hungarian',
 'indonesian',
 'italian',
 'kazakh',
 'nepali',
 'norwegian',
 'portuguese',
 'romanian',
 'russian',
 'slovene',
 'spanish',
 'swedish',
 'tajik',
 'tamil',
 'turkish',
 'uzbek']

In [4]:
stopwords.words('english')

['a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 "he'd",
 "he'll",
 'her',
 'here',
 'hers',
 'herself',
 "he's",
 'him',
 'himself',
 'his',
 'how',
 'i',
 "i'd",
 'if',
 "i'll",
 "i'm",
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it'd",
 "it'll",
 "it's",
 'its',
 'itself',
 "i've",
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'on

In [5]:
len(stopwords.words('english'))

198

# Sentence Tokenization & Word 

In [8]:
from nltk.tokenize import sent_tokenize,word_tokenize

In [7]:
text = "I love biryani. It is very tasty! Do you like it?"
sent_tokenize(text)

['I love biryani.', 'It is very tasty!', 'Do you like it?']

In [9]:
text = "I love biryani! It is very tasty 😊"
word_tokenize(text)

['I', 'love', 'biryani', '!', 'It', 'is', 'very', 'tasty', '😊']

# Stopwords Removal

In [11]:
data = pd.DataFrame({
    "Review": [
        "I love biryani and it was delicious",
        "The biryani was not very good",
        "I did not like the taste of the food",
        "This is the best biryani that I have eaten",
        "It was an average meal and it was okay"
    ]
})

data1

,Review
0,I love biryani and it was delicious
1,The biryani was not very good
2,I did not like the taste of the food
3,This is the best biryani that I have eaten
4,It was an average meal and it was okay


In [19]:
stopwordslist=stopwords.words('english')
stopwordslist.remove('not')

In [23]:
for i,doc in enumerate(data['Review']):
    doc1=[]
    for token in word_tokenize(doc):
        if token.lower() not in stopwordslist:
            doc1.append(token)
    data.loc[i,'Review']=' '.join(doc1)

In [24]:
data

,Review
0,love biryani delicious
1,biryani not good
2,not like taste food
3,best biryani eaten
4,average meal okay


# Contractions

In [26]:
import contractions

In [28]:
text = "I can't believe it's not good"
contractions.fix(text)

'I cannot believe it is not good'

In [29]:
df = pd.DataFrame({
    "Review": [
        "I can't believe the food isn't good",
        "I'm happy with the service but it hasn't arrived",
        "They've delivered late and I don't like it",
        "It's okay but I won't order again",
        "We didn't enjoy the meal and it's overpriced"
    ]
})

df

,Review
0,I can't believe the food isn't good
1,I'm happy with the service but it hasn't arrived
2,They've delivered late and I don't like it
3,It's okay but I won't order again
4,We didn't enjoy the meal and it's overpriced


In [30]:
df['Review'].apply(lambda x: contractions.fix(x))

0                I cannot believe the food is not good
1    I am happy with the service but it has not arr...
2        They have delivered late and I do not like it
3                It is okay but I will not order again
4       We did not enjoy the meal and it is overpriced
Name: Review, dtype: str

# acronyms of chats

In [31]:
df = pd.DataFrame({
    "Chat_Text": [
        "LOL that biryani was amazing 😂",
        "BRB ordering food rn",
        "IDK why the delivery is late",
        "OMG this app is so slow",
        "FYI the order was canceled smh"
    ]
})

df


,Chat_Text
0,LOL that biryani was amazing 😂
1,BRB ordering food rn
2,IDK why the delivery is late
3,OMG this app is so slow
4,FYI the order was canceled smh


In [32]:
acronyms = {
    "LOL": "laugh out loud",
    "BRB": "be right back",
    "RN": "right now",
    "IDK": "i do not know",
    "OMG": "oh my god",
    "FYI": "for your information",
    "SMH": "shaking my head"
}

In [36]:
for i,doc in enumerate(df['Chat_Text']):
    doc1=[]
    for token in word_tokenize(doc):
        if token in acronyms:
            doc1.append(acronyms[token])
        else:
            doc1.append(token)
    df.loc[i,'Chat_Text']=' '.join(doc1)

In [37]:
df

,Chat_Text
0,laugh out loud that biryani was amazing 😂
1,be right back ordering food rn
2,i do not know why the delivery is late
3,oh my god this app is so slow
4,for your information the order was canceled smh


# Stemming

In [46]:
data = {
    "Sentence": [
        "The children are playing in the gardens.",
        "She studies harder than her brothers.",
        "They were running towards the buses.",
        "He played better matches last year.",
        "The cats were chasing the mice."
    ]
}

df = pd.DataFrame(data)

print(df)


                                   Sentence
0  The children are playing in the gardens.
1     She studies harder than her brothers.
2      They were running towards the buses.
3       He played better matches last year.
4           The cats were chasing the mice.


# PorterStemmer

In [47]:
from nltk.stem import PorterStemmer

In [48]:
ps=PorterStemmer()

In [49]:
ps.stem('Programmer')

'programm'

In [50]:
ps.stem('running')

'run'

In [51]:
ps.stem('Programming')

'program'

In [52]:
ps.stem('Happily')

'happili'

In [53]:
for i,doc in enumerate(df['Sentence']):
    doc1=[]
    for token in word_tokenize(doc):
        doc1.append(ps.stem(token))
    df.loc[i,'Sentence']=' '.join(doc1)

In [54]:
df

,Sentence
0,the children are play in the garden .
1,she studi harder than her brother .
2,they were run toward the buse .
3,he play better match last year .
4,the cat were chase the mice .


# SnowballStemmer

In [11]:
from nltk.stem import SnowballStemmer

In [13]:
ss=SnowballStemmer(language='english')

In [14]:
ss.stem('Programmer')

'programm'

In [15]:
ss.stem('running')

'run'

In [16]:
ss.stem('Programming')

'program'

In [17]:
ss.stem('Happily')

'happili'

In [56]:
data = {
    "Sentence": [
        "The children are playing in the gardens.",
        "She studies harder than her brothers.",
        "They were running towards the buses.",
        "He played better matches last year.",
        "The cats were chasing the mice."
    ]
}

df = pd.DataFrame(data)
print(df)
for i,doc in enumerate(df['Sentence']):
    doc1=[]
    for token in word_tokenize(doc):
        doc1.append(ss.stem(token))
    df.loc[i,'Sentence']=' '.join(doc1)
print(df)

                                   Sentence
0  The children are playing in the gardens.
1     She studies harder than her brothers.
2      They were running towards the buses.
3       He played better matches last year.
4           The cats were chasing the mice.
                                Sentence
0  the children are play in the garden .
1    she studi harder than her brother .
2        they were run toward the buse .
3       he play better match last year .
4          the cat were chase the mice .


# LancasterStemmer

In [19]:
from nltk.stem import LancasterStemmer

In [20]:
ls=LancasterStemmer()

In [21]:
ls.stem('Programmer')

'program'

In [22]:
ls.stem('Running')

'run'

In [23]:
ls.stem('Programming')

'program'

In [24]:
ls.stem('Happily')

'happy'

In [57]:
data = {
    "Sentence": [
        "The children are playing in the gardens.",
        "She studies harder than her brothers.",
        "They were running towards the buses.",
        "He played better matches last year.",
        "The cats were chasing the mice."
    ]
}

df = pd.DataFrame(data)
print(df)
for i,doc in enumerate(df['Sentence']):
    doc1=[]
    for token in word_tokenize(doc):
        doc1.append(ls.stem(token))
    df.loc[i,'Sentence']=' '.join(doc1)
print(df) 

                                   Sentence
0  The children are playing in the gardens.
1     She studies harder than her brothers.
2      They were running towards the buses.
3       He played better matches last year.
4           The cats were chasing the mice.
                           Sentence
0  the childr ar play in the gard .
1   she study hard than her broth .
2     they wer run toward the bus .
3     he play bet match last year .
4        the cat wer chas the mic .


# Lematization

In [25]:
from nltk.stem import WordNetLemmatizer

In [26]:
wl=WordNetLemmatizer()

In [29]:
wl.lemmatize('cats') 

'cat'

In [30]:
wl.lemmatize('running',pos='n')

'running'

In [32]:
wl.lemmatize('better',pos='a')

'good'

In [35]:
wl.lemmatize('faithfully')

'faithfully'

In [58]:
data = {
    "Sentence": [
        "The children are playing in the gardens.",
        "She studies harder than her brothers.",
        "They were running towards the buses.",
        "He played better matches last year.",
        "The cats were chasing the mice."
    ]
}

df = pd.DataFrame(data)
print(df)
for i,doc in enumerate(df['Sentence']):
    doc1=[]
    for token in word_tokenize(doc):
        doc1.append(wl.lemmatize(token))
    df.loc[i,'Sentence']=' '.join(doc1)
print(df) 

                                   Sentence
0  The children are playing in the gardens.
1     She studies harder than her brothers.
2      They were running towards the buses.
3       He played better matches last year.
4           The cats were chasing the mice.
                                Sentence
0  The child are playing in the garden .
1    She study harder than her brother .
2    They were running towards the bus .
3     He played better match last year .
4       The cat were chasing the mouse .


# Data Preprocessing Function

In [236]:
def pre_processing(data,column,case='lower',tags=True,urls=True,mentions_hashtags=True,digits=True,dates=True,Contractions=True,stop_wordss=True,inflections='stem',stems='porter',emoji=True,emails=True,punctuations=True):
    import emoji
    import contractions
    from nltk.corpus import stopwords
    from nltk.tokenize import word_tokenize
    import pandas
    from nltk.stem import PorterStemmer,SnowballStemmer,LancasterStemmer,WordNetLemmatizer
    import re
    ps=PorterStemmer()
    ss=SnowballStemmer('english')
    ls=LancasterStemmer()
    wl=WordNetLemmatizer()
    
    stop_words=stopwords.words('english')
    stop_words.remove('not')

    

    if emoji==True:
        data[column]=data[column].apply(lambda x:emoji.demojize(x,delimiters=('','')))
    
    if case=='lower':
        data[column]=data[column].str.lower()
    elif case == 'upper':
        data[column]=data[column].str.upper()
    else:
        pass

    if tags==True:
        data[column]=data[column].apply(lambda x: re.sub(r'<.*?>',' ',x))

    if urls==True:
        data[column]=data[column].apply(lambda x:re.sub(r'https?://\S*','',x))

    if mentions_hashtags==True:
        data[column]=data[column].apply(lambda x: re.sub(r'\B[@#]\S+','',x))
        
    if dates==True:
        data[column]=data[column].apply(lambda x: re.sub(r'\d{1,2}-\d{1,2}-\d{4}',' ', x))
        data[column]=data[column].apply(lambda x: re.sub(r'\d{1,2}/\d{1,2}/\d{4}',' ', x))

    if digits==True:
        data[column]=data[column].apply(lambda x: re.sub(r'\d','',x))

    if Contractions==True:
        data[column]=data[column].apply(lambda x: contractions.fix(x))


    if emails==True:
        data[column]=data[column].apply(lambda x: re.sub(r'\S+@\S+','',x))

    def stop(data,column):
        l2=[]
        for doc in data[column]:
            l1=[]
            for token in word_tokenize(doc):
                if token.lower() not in stop_words:
                    l1.append(token)
            l2.append(' '.join(l1))
        return l2
    if stop_wordss==True:
        data[column]=stop(data,column)


    if inflections=='stem':
        for i,doc in enumerate(data[column]):
            l1=[]
            for token in word_tokenize(doc):
                if stems=='ps':
                    word=ps.stem(token)
                    l1.append(word)
                elif stems=='ss':
                    word=ss.stem(token)
                    l1.append(word)
                elif stems=='ls':
                    word=wl.stem(token)
                    l1.append(word) 
            data.loc[i,column]=' '.join(l1)
    elif inflections=='lemma':
        for i,doc in enumerate(data[column]):
            l1=[]
            for token in word_tokenize(doc):
                word=wl.lemmatize(token)
            l1.append(word)
            data.loc[i,column]=' '.join(l1)

    if punctuations==True:
        data[column]=data[column].apply(lambda x: re.sub('[!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~]','',x))
    
    return data

In [242]:
data1=pd.read_csv(r"C:\Users\masir\Downloads\Machine Learning\NLP\train (2).csv",on_bad_lines='skip',delimiter=';')

In [243]:
data1.head()

,Unnamed: 0,title,text,label
0,0,Palestinians switch off Christmas lights in Be...,"RAMALLAH, West Bank (Reuters) - Palestinians s...",1
1,1,China says Trump call with Taiwan president wo...,BEIJING (Reuters) - U.S. President-elect Donal...,1
2,2,FAIL! The Trump Organization’s Credit Score W...,While the controversy over Trump s personal ta...,0
3,3,Zimbabwe military chief's China trip was norma...,BEIJING (Reuters) - A trip to Beijing last wee...,1
4,4,THE MOST UNCOURAGEOUS PRESIDENT EVER Receives ...,There has never been a more UNCOURAGEOUS perso...,0


In [248]:
simple_eda(data1,'text')

The text contains the both lower and upper cases
The text contains the xml or html tags
The text contains urls
The text contains emails
The text contains mentions and hastags
The text contains emojis
The text contains digits
The text contains Punctuations
The text contains date
The text has 1 duplicate values


In [249]:
pre_processing(data1,'text')

,Unnamed: 0,title,text,label
0,0,Palestinians switch off Christmas lights in Be...,,1
1,1,China says Trump call with Taiwan president wo...,,1
2,2,FAIL! The Trump Organization’s Credit Score W...,,0
3,3,Zimbabwe military chief's China trip was norma...,,1
4,4,THE MOST UNCOURAGEOUS PRESIDENT EVER Receives ...,,0
...,...,...,...,...
24348,24348,Mexico Senate committee OK's air transport dea...,,1
24349,24349,BREAKING: HILLARY CLINTON’S STATE DEPARTMENT G...,,0
24350,24350,trump breaks from stump speech to admire beaut...,,0
24351,24351,NFL PLAYER Delivers Courageous Message: Stop B...,,0
